In [3]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tiktoken
from tqdm import tqdm
import time
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import ollama
from langchain_ollama import OllamaLLM
import ast
import pickle
import networkx as nx

In [4]:
%run ../utils/fine_tuning_util.ipynb
%run ../utils/save_and_load_util.ipynb

In [5]:
model = SentenceTransformer('all-MiniLM-L6-v2')  # Efficient embedding model
chunked_df = pd.read_csv("chunked_law_cases.csv")

In [6]:
chunked_df['embeddings'] = chunked_df['text'].apply(lambda x: model.encode(x))

In [7]:
G = nx.Graph()

In [8]:
for _, row in chunked_df.iterrows():
    G.add_node(row['chunk_id'], text=row['text'])
    if _ > 0:
        G.add_edge(row['chunk_id'], row['chunk_id'] - 1)  # Sequential linking

In [9]:
embeddings = np.array(chunked_df['embeddings'].tolist()).astype('float32')
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

In [10]:
def retrieve_similar(query):
    query_embedding = model.encode([query]).astype('float32')
    D, I = index.search(query_embedding, 5)  # Retrieve top 5 similar chunks
    return chunked_df.iloc[I[0]]['text'].tolist()

In [11]:
llm = OllamaLLM(model="llama3.1")

In [12]:
def query_llama(query):
    relevant_chunks = retrieve_similar(query)
    context = "\n".join(relevant_chunks)

    response = ollama.chat(model='llama3.1', messages=[
        {"role": "system", "content": "You are a legal AI assistant trained on case law."},
        {"role": "user", "content": f"Use this context:\n{context}\n\nQuestion: {query}"}
    ])
    return response['message']['content']

print(query_llama("What is the legal precedent for contract breaches?"))

According to the provided text, the legal precedent for contract breaches is as follows:

* A breach of contract gives the injured party a right to either compel the guilty party to perform their promise (specific performance) or sue them for damages.
* Even a breach sufficient to effect a discharge does not itself discharge the contract, but merely gives the other party an option to decide whether they will treat the contract as discharged.
* The injured party may sue for damages at once without awaiting the date fixed for performance and in the case of an obligation entitling them to specific performance, they may ask for this relief.

This is based on the statements made by Weeramantry (Vol. II, p. 884) who says that even a breach sufficient to effect a discharge does not itself discharge the contract, but merely gives the other party an option to decide whether he will treat the contract as discharged. Should they elect to do so, he may sue for damages at once without awaiting the 

In [13]:
def expand_graph_rag(query):
    similar_chunks = retrieve_similar(query)
    expanded_context = set(similar_chunks)

    for chunk in similar_chunks:
        neighbors = G.neighbors(chunked_df[chunked_df['text'] == chunk]['chunk_id'].values[0])
        expanded_context.update([G.nodes[n]['text'] for n in neighbors])

    return "\n".join(expanded_context)

In [15]:
def query_llama_graph_rag(query):
    context = expand_graph_rag(query)

    response = ollama.chat(model='llama3.1', messages=[
        {"role": "system", "content": "You are a legal AI assistant trained on case law using Graph RAG."},
        {"role": "user", "content": f"Use this context:\n{context}\n\nQuestion: {query}"}
    ])
    return response['message']['content']

print(query_llama_graph_rag("What are the implications of case law on intellectual property?"))

The provided text appears to be a court judgment or opinion rather than a straightforward question-and-answer format. However, I can extract relevant points related to intellectual property from it and provide an answer in a Q&A style based on those points.

**Q: What are the implications of case law on intellectual property?**

A: The case law discussed in this text emphasizes the importance of clearly framing issues during a trial for the court to determine. In the context of intellectual property, specifically under section 9 and section 10(1)(a) of the Intellectual Property Act, it implies that plaintiffs must ensure their cause of action includes specific claims regarding infringement rights.

**Q: What happens if the plaintiffs' pleadings do not explicitly state an issue regarding the defendant's infringement under section 9?**

A: The court may not consider or grant declarations related to such infringement even if mentioned in the plaint, as emphasized by De Silva CJ. This high